TO DO

* refactor validation and test steps
* probably need the finbert model in training mode too when training?
* extend for multi-class classification - change loss function etc.
* keeping LSTM for classification?
* needs some hyperparameter tuning

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%pip install datasets
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor
import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
from statistics import mean
import pickle
from sklearn.preprocessing import LabelEncoder


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from torch.optim import SGD

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert = AutoModel.from_pretrained("ProsusAI/finbert").to(device)
# bert = AutoModel.from_pretrained("bert-base-uncased").to(device)

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# distilbert = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

SET THE MODEL

In [6]:
model = finbert

# Data loading and cleaning

In [7]:
dataset = pd.read_csv('data_nlp.csv')
dataset

,symbol,publish_time,title,body,returns,trend
0,A,2023-05-21,"Costco, JPMorgan, Snowflake, Ford, Zoom, and M...","Earnings reports from Zoom Video, Lowe’s, Snow...",-0.162948,stable
1,A,2023-05-24,"Agilent (A) Q2 Earnings Match Estimates, Reven...",Agilent Technologies A reported second-quarter...,-5.620333,decrease
2,A,2023-06-13,Is it a Good Time to Invest in Agilent (A) Sha...,"Baron Funds, an investment management firm, re...",1.223213,stable
3,A,2023-11-20,Agilent Reports Fourth-Quarter Fiscal Year 202...,Revenue at the high end of guidance and EPS ex...,7.344231,increase
4,AAL,2023-05-22,The Undoing of American and JetBlue’s Northeas...,American Airlines CEO Robert Isom (left) speak...,-3.171385,decrease
...,...,...,...,...,...,...
3396,ZTS,2023-06-04,Zoetis' (NYSE:ZTS) five-year earnings growth t...,Stock pickers are generally looking for stocks...,0.337446,stable
3397,ZTS,2023-06-15,Zoetis Releases 2022 Sustainability Report to ...,$7.4 million in corporate giving invested in c...,3.972232,increase
3398,ZTS,2023-06-16,Zoetis’s Wetteny Joseph—one of the few Black C...,Good morning.“I was learning accounting about ...,-1.697479,stable
3399,ZTS,2023-06-21,What Makes Zoetis (ZTS) an Attractive Investme...,"Aristotle Atlantic Partners, LLC, an investmen...",1.030129,stable


In [8]:
nans = dataset[~dataset['trend'].isin(['stable', 'decrease', 'increase'])]
nans.groupby(by='symbol').count()

,publish_time,title,body,returns,trend
symbol,,,,,
ABEO,2,2,2,0,0
ADBE,38,38,38,0,0
ATI,2,2,2,0,0
ATXI,2,2,2,0,0
AVXL,2,2,2,0,0
BFAM,2,2,2,0,0
BRKL,2,2,2,0,0
CHTR,20,20,20,0,0
DUOL,1,1,1,0,0


In [9]:
len(nans)

133

In [36]:
df = dataset[['symbol', 'title', 'body', 'trend']]

In [61]:
# df.dropna(inplace = True)
# df

,symbol,title,body,trend
0,A,"Costco, JPMorgan, Snowflake, Ford, Zoom, and M...","Earnings reports from Zoom Video, Lowe’s, Snow...",stable
1,A,"Agilent (A) Q2 Earnings Match Estimates, Reven...",Agilent Technologies A reported second-quarter...,decrease
2,A,Is it a Good Time to Invest in Agilent (A) Sha...,"Baron Funds, an investment management firm, re...",stable
3,A,Agilent Reports Fourth-Quarter Fiscal Year 202...,Revenue at the high end of guidance and EPS ex...,increase
4,AAL,The Undoing of American and JetBlue’s Northeas...,American Airlines CEO Robert Isom (left) speak...,decrease
...,...,...,...,...
3396,ZTS,Zoetis' (NYSE:ZTS) five-year earnings growth t...,Stock pickers are generally looking for stocks...,stable
3397,ZTS,Zoetis Releases 2022 Sustainability Report to ...,$7.4 million in corporate giving invested in c...,increase
3398,ZTS,Zoetis’s Wetteny Joseph—one of the few Black C...,Good morning.“I was learning accounting about ...,stable
3399,ZTS,What Makes Zoetis (ZTS) an Attractive Investme...,"Aristotle Atlantic Partners, LLC, an investmen...",stable


In [40]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_smaller_dataset(df):

  df = df.dropna()

  sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

  X = df.drop('trend', axis=1)
  y = df['trend']

  for train_index, test_index in sss.split(X, y):
      X_sample, y_sample = X.iloc[test_index], y.iloc[test_index]

  sample_df = X_sample.assign(trend=y_sample.values)
  sample_df.reset_index(inplace=True, drop=True)

  return sample_df

In [41]:
df = get_smaller_dataset(df)

# Old data

In [127]:
# dataset = load_dataset("edarchimbaud/news-stocks")
# dataset.set_format(type='pandas')

# df = dataset['train'][:]
# df

In [128]:
# df = df.drop(['publisher', 'url', 'uuid'], axis=1)
# df

In [129]:
# # select stocks
# ticker = 'AAPL'

# df = df[df['symbol'] == ticker]

# df = df.reset_index(drop=True)
# df

# Load stock prices

In [130]:
# # start date is 7 days before the earliest day from the df
# # to make sure at least one trading day before is included
# # (there might be weekend days/holidays etc.)

# start_date = df['publish_time'].min() - datetime.timedelta(7)
# end_date = df['publish_time'].max() + datetime.timedelta(7)

# prices = yf.download('AAPL', start_date, end_date)

# prices.index = pd.to_datetime(prices.index, format='%Y-%m-%d', utc=True)
# prices

In [131]:
# def trend(date):

#   prev_date = prices[prices.index < date].index.max()
#   prev_date_index = prices.index.get_loc(prev_date.strftime('%Y-%m-%d'))

#   if date.strftime('%Y-%m-%d') == prev_date.strftime('%Y-%m-%d'):
#     prev_date_index -= 1

#   next_date = prices[prices.index > date].index.min()
#   next_date_index = prices.index.get_loc(next_date.strftime('%Y-%m-%d'))
#   ret = ((prices['Open'][next_date_index] - prices['Close'][prev_date_index]) / prices['Close'][prev_date_index]) * 100

#   return_threshold = 1.0 # (1%)
#   if ret >= return_threshold:
#     return 'increase'
#   elif ret <= -return_threshold:
#     return 'decrease'
#   else:
#     return 'stable'


# df['trend'] = df['publish_time'].apply(trend)

# Tokenize data

In [42]:
def wrap_tokenizer(tokenizer, padding=True, truncation=True, return_tensors='pt', max_length=None):
    def tokenize(text):
        text = list(text)
        tokens = tokenizer(
            text,
            padding=padding,
            return_attention_mask=False,
            truncation=truncation,
            max_length=max_length,
            return_tensors=return_tensors
            )['input_ids']
        return tokens
    return tokenize

In [43]:
tokenize = wrap_tokenizer(tokenizer, padding=False, truncation = False, return_tensors=None)
tokens = tokenize(df['body'])
num_tokens = [len(x) for x in tokens]
df['length'] = pd.Series(num_tokens)

In [44]:
df.head()

,symbol,title,body,trend,length
0,KMX,Used-car market in ‘uncharted territory’ as hi...,"With spring car-buying season in full swing, t...",increase,752
1,CI,Strong Forecast from UnitedHealth Sends Insure...,UnitedHealth Group boosted its profit outlook....,increase,92
2,FEMY,Femasys Inc. Secures Additional Financing with...,Femasys Inc.- $6.85 million upfront investment...,increase,2871
3,MMM,U.S. News & World Report incorporates 3M Ambul...,"ST. PAUL, Minn., July 13, 2023 /PRNewswire/ --...",stable,1715
4,TTI,TETRA Technologies Insider Buyers Net US$98k D...,"Insiders who bought TETRA Technologies, Inc. (...",decrease,799


In [15]:
# df = df.iloc[:50]
# df.dropna(inplace=True)

In [45]:
max_length = df['length'].unique().max()
max_length

16358

In [46]:
max_length = df['length'].unique().max()
max_length
max_tokens = 512
for i in range(1,(max_length//max_tokens)):
    num = sum(df['length']>i*max_tokens)
    print(f"Number of text that have more than {i}*max_tokens is {num}")

Number of text that have more than 1*max_tokens is 662
Number of text that have more than 2*max_tokens is 429
Number of text that have more than 3*max_tokens is 275
Number of text that have more than 4*max_tokens is 197
Number of text that have more than 5*max_tokens is 148
Number of text that have more than 6*max_tokens is 100
Number of text that have more than 7*max_tokens is 73
Number of text that have more than 8*max_tokens is 53
Number of text that have more than 9*max_tokens is 38
Number of text that have more than 10*max_tokens is 34
Number of text that have more than 11*max_tokens is 24
Number of text that have more than 12*max_tokens is 19
Number of text that have more than 13*max_tokens is 16
Number of text that have more than 14*max_tokens is 10
Number of text that have more than 15*max_tokens is 10
Number of text that have more than 16*max_tokens is 9
Number of text that have more than 17*max_tokens is 7
Number of text that have more than 18*max_tokens is 5
Number of text t

In [47]:
def get_text_split(text, length=200, overlap=50, max_chunks=4):
    chunks = []

    words = text.split()
    n_words = len(words)

    n = max(1, min(max_chunks, (n_words - length) // (length - overlap) + 1))

    for i in range(n):
        start_idx = i * (length - overlap)
        end_idx = min(start_idx + length, n_words)

        chunk_words = words[start_idx:end_idx]

        chunk_text = " ".join(chunk_words)

        # If it's the last chunk and its length is less than 75% of the desired length, skip
        if i == n - 1 and len(chunk_words) < 0.75 * length and n > 1:
            continue

        chunks.append(chunk_text)

    return chunks

**ENCODE LABELS**

In [48]:
labels = ["increase", "stable", "decrease"]

label_encoder = LabelEncoder()

df['trend'] = label_encoder.fit_transform(df['trend'])

# Train-val-test split

In [49]:
def split_df(df):
    n_rows = len(df)

    df_train = df.iloc[:int(0.8*n_rows),:]
    df_val = df.iloc[int(0.8*n_rows):int(0.9*n_rows),:]
    df_test = df.iloc[int(0.9*n_rows):,:]

    return df_train, df_val, df_test

In [50]:
n_rows = len(df)
dfs_train, dfs_val, dfs_test = [],[],[]
gb = df.groupby('trend')
for x in gb.groups:
    group = gb.get_group(x)
    df_train, df_val, df_test = split_df(group)
    dfs_train.append(df_train)
    dfs_val.append(df_val)
    dfs_test.append(df_test)

df_train = pd.concat(dfs_train, ignore_index=True)

df_val = pd.concat(dfs_val, ignore_index=True)

df_test = pd.concat(dfs_test, ignore_index=True)

print(f'Number of training examples: {len(df_train)}')
print(f'Number of validation examples: {len(df_val)}')
print(f'Number of test examples: {len(df_test)}')

Number of training examples: 652
Number of validation examples: 82
Number of test examples: 83


In [51]:
df_train.body = df_train.body.apply(lambda x: get_text_split(x))
df_val.body = df_val.body.apply(lambda x: get_text_split(x))
df_test.body = df_test.body.apply(lambda x: get_text_split(x))

In [52]:
df_train['n_chunks'] = df_train.body.apply(lambda x: len(x))
df_val['n_chunks'] = df_val.body.apply(lambda x: len(x))
df_test['n_chunks'] = df_test.body.apply(lambda x: len(x))

In [53]:
df_train.head()

,symbol,title,body,trend,length,n_chunks
0,TTI,TETRA Technologies Insider Buyers Net US$98k D...,"[Insiders who bought TETRA Technologies, Inc. ...",0,799,3
1,RSG,"Republic Services, Inc. (NYSE:RSG) Delivered A...",[Many investors are still learning about the v...,0,1679,4
2,GSIT,"GSI Technology, Inc. (NASDAQ:GSIT) Q2 2024 Ear...","[GSI Technology, Inc. (NASDAQ:GSIT) Q2 2024 Ea...",0,2126,4
3,ARE,"Alexandria Real Estate Equities, Inc.'s Discip...","[PASADENA, Calif., May 24, 2023 /PRNewswire/ -...",0,1437,4
4,MO,Altria (MO) Gains on Pricing Power Amid Low Ci...,"[Altria Group, Inc. MO looks well-positioned d...",0,1328,4


# Model

HYPERPARAMETERS

In [54]:
BATCH_SIZE = 10
EPOCHS = 5
LR = 1e-7
EMBEDDING_DIM = 768 # 768 for base and 1024 for large
LSTM_SIZE = 128
LSTM_DO = 0.2
POOLED_EMB_DO = 0.3
OUT_DIM_LIN = 3
WEIGHT_DECAY = 1e-3
#=============================
# freeze xlmr hyperparams
EPOCHS_FREEZE = 2
NUM_WARMUP_EPOCHS_FREEZE = 1
LR_FREEZE = 6e-5
#=============================
LR_LIN = 3e-6
EPOCHS_LIN = 50


In [55]:
class MyDataset(Dataset):
    def __init__(self,df):
        self.n_chunks = df['n_chunks'].to_list()
        self.X = df['body'].to_list()
        self.Y = df['trend']

    def __len__(self):
        return len(self.X)

    def __getitem__(self,index):
        return self.X[index], self.Y.iloc[index], self.n_chunks[index]

def collate_func(batch):
    X = [x[0] for x in batch]
    Y = torch.Tensor([x[1] for x in batch])
    c = [x[2] for x in batch]
    return [X,Y,c]

In [56]:
class Classifier(nn.Module):
    def __init__(self, lstm_size, emb_dim, out_dim_lin, lstm_do):
        super().__init__()

        self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=lstm_size, batch_first=True)
        self.dropout = nn.Dropout(lstm_do)
        self.linear = nn.Linear(in_features=lstm_size, out_features=out_dim_lin)

    def forward(self, x, n_chunks):
        x = pad_sequence(x, batch_first=True, padding_value=0)
        x = pack_padded_sequence(input=x, lengths=n_chunks, batch_first=True, enforce_sorted=False)
        x, _ = self.lstm(x)
        x, _ = pad_packed_sequence(x, batch_first=True)
        x = x[:,-1,:]
        x = self.dropout(x)
        logit = self.linear(x)
        return logit

In [69]:
# class Classifier(nn.Module):
#     def __init__(self, bert_model, out_dim_lin, dropout):
#         super().__init__()

#         self.bert = bert_model
#         self.dropout = nn.Dropout(dropout)
#         self.linear = nn.Linear(in_features=self.bert.config.hidden_size, out_features=out_dim_lin)

#     def forward(self, x, n_chunks):

#         x = pad_sequence(x, batch_first=True, padding_value=0)

#         with torch.no_grad():
#             bert_outputs = self.bert(x)

#         bert_last_hidden_state = bert_outputs.last_hidden_state

#         x = self.dropout(bert_last_hidden_state)

#         logit = self.linear(x[:, 0, :])

#         return logit

In [70]:
# def save_to_disk(txt_path, values):
#     if os.path.isfile(txt_path):
#         os.remove(txt_path)
#     with open(txt_path, "wb") as fp:
#         pickle.dump(values, fp)
#     return


# def load_from_disk(txt_path):
#     with open(txt_path, "rb") as f:
#         values =  pickle.load(f)
#     return values


# def save_checkpoint(xlmr, classifier, optimizer, logs, checkpoint_dir, epoch):
#     print('')
#     print('Saving checkpoint...')
#     state_dict = {
#         'classifier':classifier.state_dict(),
#         'optimizer': optimizer.state_dict(),
#     }
#     torch.save(state_dict, os.path.join(checkpoint_dir, 'checkpoint_{}.pt'.format(epoch)))
#     save_to_disk(os.path.join(checkpoint_dir, 'logs.txt'),logs)
#     print(f'Checkpoint saved!')

# checkpoint_dir = '/content/google_drive/MyDrive/checkpoints'

# def load_checkpoint(checkpoint_dir, epoch, xlmr, classifier, device, optimizer=None):
#     pretrained_dict = torch.load(os.path.join(checkpoint_dir,'checkpoint_{}.pt'.format(epoch)),map_location=torch.device(device))
#     classifier.load_state_dict(pretrained_dict['classifier'])
#     if optimizer is not None:
#         optimizer.load_state_dict(pretrained_dict['optimizer'])
#         return classifier, optimizer
#     return classifier

In [71]:
# def get_lr(optimizer):
#     for param_group in optimizer.param_groups:
#         return param_group['lr']

# def set_lr(optimizer, lr):
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

In [72]:
# def train(train_loader, tokenize, device, model, optimizer, classifier, dropout=0.0):

#     classifier.train()
#     model.train()

#     accuracy = []
#     losses = []

#     for text, target, n_chunks in train_loader:
#         target = target.to(device)

#         flat_text = [item for sublist in text for item in sublist]
#         tokens = tokenize(flat_text)
#         tokens = tokens.to(device)

#         # Based on the results - I think we should finetune the model
#         # with torch.no_grad():
#         #   outputs = model(tokens)
#         outputs = model(tokens)

#         embeddings = outputs.last_hidden_state

#         pooled_emb = torch.mean(embeddings, axis=1)
#         pooled_emb = nn.Dropout(dropout)(pooled_emb)
#         x = [s for s in torch.split(pooled_emb, n_chunks, dim=0)]

#         logit = classifier(x, n_chunks)
#         prob = torch.sigmoid(logit)

#         target = torch.reshape(target, shape=(-1, 1))
#         loss = nn.BCELoss()(input=prob, target=target.float())

#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         prediction = (prob >= 0.5).float()
#         correct_predictions = torch.sum(prediction == target).item()
#         acc = correct_predictions / len(target)
#         accuracy.append(acc)
#         losses.append(loss.item())

#     return accuracy, losses


In [57]:
def train(train_loader, tokenize, device, model, optimizer, classifier, dropout=0.0):
    classifier.train()
    model.train()

    accuracy = []
    losses = []

    for text, target, n_chunks in train_loader:
        print('!!!')
        target = target.long().to(device)

        flat_text = [item for sublist in text for item in sublist]
        tokens = tokenize(flat_text)
        tokens = tokens.to(device)

        # with torch.no_grad():
        #   outputs = model(tokens)

        outputs = model(tokens)
        embeddings = outputs.last_hidden_state

        pooled_emb = torch.mean(embeddings, axis=1)
        pooled_emb = nn.Dropout(dropout)(pooled_emb)
        x = [s for s in torch.split(pooled_emb, n_chunks, dim=0)]

        logits = classifier(x, n_chunks)

        loss = nn.CrossEntropyLoss()(input=logits, target=target)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        _, predictions = torch.max(logits, 1)
        correct_predictions = torch.sum(predictions == target).item()
        acc = correct_predictions / target.size(0)
        accuracy.append(acc)
        losses.append(loss.item())

    return accuracy, losses

In [58]:
def val_step(val_loader, tokenize, device, model, classifier, is_test=False):

    classifier.eval()
    model.eval()

    accuracy = []
    losses = []

    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for text, target, n_chunks in val_loader:
            target = target.to(device)

            flat_text = [item for sublist in text for item in sublist]
            tokens = tokenize(flat_text)
            tokens = tokens.to(device)

            outputs = model(tokens)
            embeddings = outputs.last_hidden_state

            pooled_emb = torch.mean(embeddings, axis=1)
            x = [s for s in torch.split(pooled_emb, n_chunks, dim=0)]

            logits = classifier(x, n_chunks)

            loss = nn.CrossEntropyLoss()(input=logits, target=target)

            _, predictions = torch.max(logits, 1)
            correct_predictions = torch.sum(predictions == target).item()
            acc = correct_predictions / target.size(0)
            accuracy.append(acc)
            losses.append(loss.item())

            if is_test:
                all_predictions.append(predictions.cpu())
                all_targets.append(target.cpu())

    if is_test:
        all_predictions = torch.cat(all_predictions).numpy()
        all_targets = torch.cat(all_targets).numpy()
        return accuracy, losses, all_predictions, all_targets

    return accuracy, losses

In [59]:
train_dataset = MyDataset(df_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_func)

val_dataset = MyDataset(df_val)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_func)

test_dataset = MyDataset(df_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_func)

# Sentiment Analysis

In [60]:
import os

# checkpoint_dir = '/content/google_drive/MyDrive/checkpoints'
# os.makedirs(checkpoint_dir, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Selected device is {}'.format(device))

classifier = Classifier(lstm_size=LSTM_SIZE,
                        emb_dim=EMBEDDING_DIM,
                        out_dim_lin=OUT_DIM_LIN,
                        lstm_do = LSTM_DO).to(device)


params = list(model.parameters()) + list(classifier.parameters())
#params = list(classifier.parameters())
# optimizer = SGD(params, lr=LR_FREEZE, weight_decay=WEIGHT_DECAY)
optimizer = AdamW(params, lr=5e-5)

tokenize = wrap_tokenizer(tokenizer)

Selected device is cpu


In [61]:
logs = {'train_acc':[],'train_loss':[],
        'val_acc':[],'val_loss':[]}

for epoch in range(EPOCHS):

    train_acc, train_loss = train(train_loader = train_loader,
                                  tokenize = tokenize,
                                  device = device,
                                  optimizer = optimizer,
                                  model = model,
                                  classifier = classifier,
                                  dropout=POOLED_EMB_DO)

    # val_acc, val_loss, val_cm = val_step(
    #     val_loader=val_loader,
    #     tokenize=tokenize,
    #     device=device,
    #     xlmr=xlmr,
    #     classifier=classifier
    # )

    logs['train_acc'] += train_acc
    logs['train_loss'] += train_loss
    # logs['val_acc'] += val_acc
    # logs['val_loss'] += val_loss

    # if epoch % 2 == 0 and epoch != 0 :
    #     save_checkpoint(
    #         xlmr=xlmr,
    #         classifier=classifier,
    #         optimizer=optimizer,
    #         logs=logs,
    #         checkpoint_dir=checkpoint_dir,
    #         epoch=epoch
    #         )


    print(f"Epoch {epoch} --> loss:{mean(train_loss):.4f},\
                               acc:{mean(train_acc): .4f}% ")

!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
Epoch 0 --> loss:0.7820,                               acc: 0.6379% 
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!
!!!


KeyboardInterrupt: 

FinBert (training)

* Epoch 0 --> loss:0.6814,                               acc: 0.6089%
* Epoch 1 --> loss:0.6841,                               acc: 0.5881%
* Epoch 2 --> loss:0.6828,                               acc: 0.6600%
* Epoch 3 --> loss:0.6844,                               acc: 0.5881%


Distil Bert (training)

* Epoch 0 --> loss:0.6897,                               acc: 0.5028%
* Epoch 1 --> loss:0.6923,                               acc: 0.4934%
* Epoch 2 --> loss:0.6905,                               acc: 0.5038%
* Epoch 3 --> loss:0.6934,                               acc: 0.4839%
* Epoch 4 --> loss:0.6882,                               acc: 0.5350%